<a href="https://colab.research.google.com/github/dong-han-lee/D4SG_Dash_WEB/blob/main/D4SG_Dash_WEB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安裝套件
!pip install -q dash
!pip install -q dash-bootstrap-components

In [ ]:
# 載入相關套件
import dash
import dash_bootstrap_components as dbc
from dash import Input, Output, State, dcc, html

import pandas as pd
import numpy as np
import plotly.express as px

import os
import json
import gdown

In [ ]:
# 讀取量能服務狀況資料
file_id = '1gCCcerBTKrHtfPaTJlO3JVVM6D-xT4PmQRlpA7zS4kI'
gdrive_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(gdrive_url, '服務量能狀況.xlsx', quiet=True)

# 讀取資料
serviceData = pd.read_excel('服務量能狀況.xlsx', sheet_name="WEB")
serviceData = serviceData.round(2)  # 數值四捨五入
serviceData = serviceData.fillna(0)  # 缺值補0

In [ ]:
# 選項清單
mapOptions = serviceData.drop(columns=['居住鄉鎮']).columns

In [ ]:
# 讀取鄉鎮市區界線(TWD97經緯度)資料
file_id = '11mf-In-JYqShNUICh7BVa2loLJqmliWD'
gdrive_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(gdrive_url, 'TOWN_MOI_1120825.geojson', quiet=True)
with open(f"TOWN_MOI_1120825.geojson", encoding='utf8') as response:
  mapGeo = json.load(response)

# 篩選屏東縣鄉鎮市區清單
mapGeo['features'] = [elem for elem in mapGeo['features'] if elem['properties']['COUNTYNAME'] == '屏東縣']

# 依據圖資資料建立鄉鎮市區清單
mapDf = pd.DataFrame.from_dict([i['properties'] for i in mapGeo['features']])
mapDf['居住鄉鎮'] = mapDf['COUNTYNAME'] + mapDf['TOWNNAME']

In [ ]:
# 將個管系統統計資訊併入地圖資料
mapDf = mapDf.merge(serviceData, on=['居住鄉鎮'], how='left')

In [ ]:
# # 繪製屏東縣各鄉鎮市個案數地圖
# fig = px.choropleth_mapbox(mapDf,  # 資料表
#                            geojson=mapGeo,  # 地圖資訊
#                            locations='TOWNCODE',  # df要對應geojson的id名稱
#                            featureidkey='properties.TOWNCODE',  # geojson對應df的id名稱
#                            color='個案數',  # 顏色區分對象
#                            color_continuous_scale='Viridis',  # 設定呈現的顏色
#                            range_color=(round(np.nanmin(mapDf['個案數'])),  # 顏色的值域範圍
#                                         round(np.nanmax(mapDf['個案數']))),
#                            mapbox_style='carto-positron',  # mapbox地圖格式
#                            zoom=8.5,  # 地圖縮放大小: 數字愈大放大程度愈大
#                            center={'lat': 22.4, 'lon': 120.49138},  # 地圖中心位置
#                            opacity=0.5,  # 設定顏色區塊的透明度 數值愈大愈不透明
#                            hover_data=['居住鄉鎮', '個案數'],  # 設定游標指向資訊
#                            width=800,
#                            height=600,
#                           )
# fig.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})
# fig.show()

In [ ]:
# 建立css檔案
css_content = """
  body {
      font-family: sans-serif;
  }
"""

os.makedirs('assets', exist_ok=True)
with open('assets/styles.css', 'w') as file:
  file.write(css_content)

In [ ]:
# Dash設定
app = dash.Dash(
    external_stylesheets=[dbc.themes.BOOTSTRAP]
)

# 網站名稱
app.title = 'D4SG資料英雄計畫: 屏東縣社會處長青科專案'

In [ ]:
# 服務量能儀表板
page_dashboard = dbc.Container([

    dbc.Row([
       html.Center(html.H3("服務量能儀表板")),
    ]),

    dbc.Row([

        # 選單欄位
        dbc.Col([
          html.Span("請選擇繪製指標:"),
          dcc.Dropdown(
            id='mapDropdown',
            options={elem: elem for elem in mapOptions},
            value=mapOptions[0],
          )
          ], md=3),
        ], className="mb-5"),

    dbc.Row([
        # 繪製地圖欄位
        dbc.Col([
            dbc.Spinner(html.Center(dcc.Graph(id='map')), color="primary"),
        ]),
    ]),
])


# 建立地圖
@app.callback(
    Output("map", "figure"),
    Input(f"mapDropdown", "value"),
    )
def MakeMapFig(selected_name):

  # 繪製屏東縣各鄉鎮市個案數地圖
  fig = px.choropleth_mapbox(mapDf,  # 資料表
                            geojson=mapGeo,  # 地圖資訊
                            locations='TOWNCODE',  # df要對應geojson的id名稱
                            featureidkey='properties.TOWNCODE',  # geojson對應df的id名稱
                            color=selected_name,  # 顏色區分對象
                            color_continuous_scale='Viridis',  # 設定呈現的顏色
                            range_color=(round(np.nanmin(mapDf[selected_name])),  # 顏色的值域範圍
                                          round(np.nanmax(mapDf[selected_name]))),
                            mapbox_style='carto-positron',  # mapbox地圖格式
                            zoom=8.5,  # 地圖縮放大小: 數字愈大放大程度愈大
                            center={'lat': 22.4, 'lon': 120.49138},  # 地圖中心位置
                            opacity=0.5,  # 設定顏色區塊的透明度 數值愈大愈不透明
                            hover_data=['居住鄉鎮']+[elem for elem in mapOptions if elem != selected_name],  # 設定游標指向資訊
                            width=800,
                            height=600,
                            )
  fig.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})

  return fig

In [ ]:
# 前端畫面版型
app.layout = html.Div([

    # navbar
    dbc.Navbar(
        dbc.Container(
            [
                html.A(
                    # Use row and col to control vertical alignment of logo / brand
                    dbc.Row(
                        [
                            dbc.Col(dbc.NavbarBrand("D4SG: 屏東縣社會處長青科專案", className="ml-2")),
                        ],
                        align="center",
                        className="g-0",
                    ),
                    href="/",
                    style={"textDecoration": "none"},
                ),
                dbc.NavbarToggler(id="navbar-toggler-menu"),
                dbc.Collapse(
                    dbc.Nav(
                        [
                            dbc.NavItem(dbc.NavLink("服務量能儀表板", href="/page_dashboard", active="exact")),
                        ],
                        className="ml-2", navbar=True),
                    id="navbar-collapse-menu",
                    navbar=True,
                ),
            ]
        ),
        color="dark",
        dark=True,
        className="mb-5",
    ),

    # page content
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])


# toggle the collapse on small screens
@app.callback(
    Output(f"navbar-collapse-menu", "is_open"),
    [Input(f"navbar-toggler-menu", "n_clicks")],
    [State(f"navbar-collapse-menu", "is_open")])
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


# navbar link
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):

    if pathname == "/":
        return page_dashboard
    elif pathname == "/page_dashboard":
            return page_dashboard

    # If the user tries to reach a different page, return a 404 message
    return html.Div(
        dbc.Container([
            html.H3("404: 找不到您想要的頁面...", className="display-3"),
            html.Hr(className="my-2"),
            html.P(
                f"The pathname {pathname} was not recognised..."
                ),
            ], fluid=True, className="py-3",),
        className="p-3 bg-light rounded-3",
    )

In [ ]:
if __name__ == "__main__":
    app.run_server(jupyter_mode="external")

Dash app running on:


<IPython.core.display.Javascript object>